In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'
PATH = "/home/xjaros2/Documents/git/csidh-setup/csidh-target/src/"
attack_type = "A2_CALL"

In [2]:
from csidh import CSIDHCW
import chipwhisperer as cw
import random

In [3]:
cw.set_all_log_levels(cw.logging.CRITICAL)

In [4]:
csidh = CSIDHCW(PATH, attack_type=attack_type, PLATFORM=PLATFORM)
csidh.name = 'Lite'
csidh.setup()

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 1733463352                to 1773396507               
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 96000000                  to 5984607                  
scope.clock.adc_rate                     changed from 96000000.0                to 5984607.0         

In [5]:
csidh.flash_target()

SS_VER set to SS_VER_2_1
.
-------- begin --------
.
Cleaning project:
rm -f -- main-CW308_CC2538.hex main-CW301_AVR.hex main-CW303.hex main-CW304.hex main-CW308_MEGARF.hex main-CW308_SAM4L.hex main-CW308_STM32F0.hex main-CW308_STM32F1.hex main-CW308_STM32F2.hex main-CW308_STM32F3.hex main-CW308_STM32F4.hex main-CW308_K24F.hex main-CW308_NRF52.hex main-CW308_AURIX.hex main-CW308_SAML11.hex main-CW308_EFM32TG11B.hex main-CWLITEARM.hex main-CWLITEXMEGA.hex main-CWNANO.hex main-CW308_K82F.hex main-CW308_PSOC62.hex main-CW308_IMXRT1062.hex main-CW308_FE310.hex main-CW308_EFR32MG21A.hex main-CW308_EFM32GG11.hex main-CW308_STM32L5.hex main-CW308_NEORV32.hex main-CW308_SAM4S.hex main-CW305_IBEX.hex
rm -f -- main-CW308_CC2538.eep main-CW301_AVR.eep main-CW303.eep main-CW304.eep main-CW308_MEGARF.eep main-CW308_SAM4L.eep main-CW308_STM32F0.eep main-CW308_STM32F1.eep main-CW308_STM32F2.eep main-CW308_STM32F3.eep main-CW308_STM32F4.eep main-CW308_K24F.eep main-CW308_NRF52.eep main-CW308_AURIX.eep

In [6]:
csidh.reset_target()
csidh.scope.arm()
csidh.action()
ret = csidh.scope.capture()
if ret:
    print("Timeout happened during acquisition")
PUBLIC_EXPECTED = csidh.public_with_errors
max_ext_offset = csidh.scope.adc.trig_count
print("Public key:", PUBLIC_EXPECTED)
print("Max ext offset:", max_ext_offset)

Public key: 0
Max ext offset: 10058044


In [7]:
# csidh.setup_()

In [8]:
# # Glitch setup
# csidh.scope.glitch.clk_src = "clkgen"
# csidh.scope.glitch.output = "clock_xor"
# csidh.scope.glitch.trigger_src = "ext_single"
# csidh.scope.io.hs2 = "glitch"
csidh.voltage_glitching_setup()

In [9]:
# Adc timeout
csidh.scope.adc.timeout = 0.5

In [10]:
from csidh.search import Unit, generate_population, write_cache_to_file
from collections import OrderedDict
import numpy as np
from tqdm import tqdm

cache = OrderedDict()


def evaluate_unit(csidh, unit, num_measurements=1):
    """Evaluates a single unit"""
    if csidh.scope._is_husky:
        csidh.scope.glitch.num_glitches = 1
        csidh.scope.glitch.width = int(unit.width)
        csidh.scope.glitch.offset = int(unit.offset)
        csidh.scope.glitch.repeat = int(unit.repeat)
        csidh.scope.glitch.ext_offset = int(unit.ext_offset)
    else:
        csidh.scope.glitch.width = unit.width
        csidh.scope.glitch.offset = unit.offset
        csidh.scope.glitch.repeat = unit.repeat
        csidh.scope.glitch.ext_offset = unit.ext_offset

    # Perform the measurements
    measurements = []
    responses = []

    for _ in range(num_measurements):
        csidh.reset_target()
        if csidh.scope._is_husky:
            csidh.scope.glitch.state = None
        csidh.scope.arm()
        ret = csidh.action()
        csidh.scope.io.vglitch_reset()
        if ret:
            logging.error("Timeout happened during acquisition")


        public_received = csidh.public_with_errors
        if not isinstance(public_received, int):
            measurements.append("RESET")
        elif public_received == PUBLIC_EXPECTED:
            measurements.append("NORMAL")
        else:
            measurements.append("JUSTRIGHT")
            responses.append(public_received)

    unit.width = csidh.scope.glitch.width  # CW rounds the values
    unit.offset = csidh.scope.glitch.offset
    unit.repeat = csidh.scope.glitch.repeat
    unit.measurements = measurements
    unit.responses = responses
    

    # Classify
    if not all(m == measurements[0] for m in measurements):
        unit.type = "CHANGING"
        N_normal = sum(1 for m in measurements if m == "NORMAL")
        N_reset = sum(1 for m in measurements if m == "RESET")
        N_justright = sum(1 for m in measurements if m == "JUSTRIGHT")
        unit.fitness = 4 + 1.2 * N_justright + 0.2 * N_normal + 0.5 * N_reset
    else:
        if measurements[0] == "NORMAL":
            unit.type = "NORMAL"
            unit.fitness = 2
        elif measurements[0] == "RESET":
            unit.type = "RESET"
            unit.fitness = 5
        elif measurements[0] == "JUSTRIGHT":
            unit.type = "JUSTRIGHT"
            unit.fitness = 10
    cache[unit] = unit.fitness
    if unit.responses != []:
        pass
    print(unit)
    print(unit.responses)


def evaluate_batch(csidh, population):
    uncached = [u for u in population if u not in cache]
    if not uncached:
        return [u.fitness for u in population]

    to_visit = np.array(uncached)

    for unit in population:
        if unit in cache:
            unit.fitness = cache[unit]

    for unit in tqdm(to_visit):
        evaluate_unit(csidh, unit)

    return [u.fitness for u in population]

In [11]:
def generate_whole_population():
    population = []
    for ext_offset in range(Unit.EXT_OFFSET_MIN, Unit.EXT_OFFSET_MAX+1, 1):
        for repeat in range(Unit.REPEAT_MIN, Unit.REPEAT_MAX+1):
            unit = Unit(repr=f"{ext_offset},{Unit.OFFSET_MIN},{Unit.WIDTH_MIN},{repeat},None,0,0,0")
            population.append(unit)
    return population

In [12]:
TOP15 = [
    (-9.0, 46.0),
    (-9.0, 44.0),
    (3.0, -13.0),
    (9.0, 3.0),
    (14.0, -3.0),
    (-13.0, -46.0),
    (-12.0, -47.0),
    (-13.0, -41.0),
    (-6.0, 42.0),
    (5.0, -13.0),
    (-13.0, -47.0),
    (5.0, 7.0),
    (-7.0, 43.0),
    (-3.0, 40.0),
    (-13.0, -43.0),
]

batch = []
REPEATS = 10000
for _ in range(REPEATS):
    i = random.randint(0, len(TOP15) - 1)
    width = TOP15[i][0]
    offset = TOP15[i][1]
    repeat = random.randint(8, 18)
    ext_offset = random.randint(2175, 2222)

    unit = Unit(repr=f"{ext_offset},{offset},{width},{repeat},None,0,0,0")
    batch.append(unit)
batch.sort(key=lambda unit:unit.ext_offset)

In [13]:
cache = OrderedDict()
evaluate_batch(csidh, batch)
write_cache_to_file(f"cwlitearm-xISOG-5-isogeny-skip-test.json", cache, 1, len(batch), -1)   

  0%|                                                                             | 1/10000 [00:00<2:10:40,  1.28it/s]

(ext_offset=2175, offset=-3.125, width=14.0625, repeat=9, type=NORMAL, fitness=2)
[]


  0%|                                                                             | 2/10000 [00:01<2:10:38,  1.28it/s]

(ext_offset=2175, offset=39.84375, width=-3.125, repeat=11, type=NORMAL, fitness=2)
[]


  0%|                                                                             | 3/10000 [00:02<2:10:37,  1.28it/s]

...

(ext_offset=2176, offset=-12.890625, width=3.125, repeat=9, type=NORMAL, fitness=2)
[]


  2%|█▋                                                                         | 232/10000 [03:02<2:07:41,  1.27it/s]

(ext_offset=2176, offset=-46.875, width=-12.890625, repeat=13, type=NORMAL, fitness=2)
[]


  2%|█▋                                                                         | 232/10000 [03:02<2:08:14,  1.27it/s]


USBErrorNoDevice: LIBUSB_ERROR_NO_DEVICE [-4]